# Benchmarks

## Initialize

In [1]:
import os
import math
import pathlib
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.feather as feather
from tqdm.auto import tqdm
from IPython.display import clear_output

import warnings
from lifelines.utils import CensoringType
from lifelines.utils import concordance_index

In [2]:
node = !hostname
if "sc" in node[0]:
    base_path = "/sc-projects/sc-proj-ukb-cvd"
else: 
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"
print(base_path)

project_label = "22_retina_phewas_220603_fullrun"
project_path = f"{base_path}/results/projects/{project_label}"
figure_path = f"{project_path}/figures"
output_path = f"{project_path}/data"

pathlib.Path(figure_path).mkdir(parents=True, exist_ok=True)
pathlib.Path(output_path).mkdir(parents=True, exist_ok=True)

experiment = '220603_fullrun'
experiment_path = f"{output_path}/{experiment}"
pathlib.Path(experiment_path).mkdir(parents=True, exist_ok=True)

name_dict = {
    "predictions_cropratio0.3": "ConvNextSmall(Retina)+MLP_cropratio0.3",
    "predictions_cropratio0.5": "ConvNextSmall(Retina)+MLP_cropratio0.5",
    "predictions_cropratio0.8": "ConvNextSmall(Retina)+MLP_cropratio0.8",
}

partitions = [i for i in range(22)]
partitions

/sc-projects/sc-proj-ukb-cvd


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [12]:
import pandas as pd
endpoints = sorted([l.replace('_prevalent', '') for l in list(pd.read_csv('/sc-projects/sc-proj-ukb-cvd/results/projects/22_retinal_risk/data/220602/endpoints.csv').endpoint.values)])

In [13]:
out_path = f"{experiment_path}/coxph/predictions"
pathlib.Path(out_path).mkdir(parents=True, exist_ok=True)

In [14]:
from sklearn.preprocessing import StandardScaler
import pickle
import zstandard

def read_data(fp_in):
    temp = pd.read_feather(f"{fp_in}").set_index("eid")
    return temp   
    
def save_pickle(data, data_path):
    with open(data_path, "wb") as fh:
        cctx = zstandard.ZstdCompressor()
        with cctx.stream_writer(fh) as compressor:
            compressor.write(pickle.dumps(data, protocol=pickle.HIGHEST_PROTOCOL))
    
def read_predictions(endpoint, feature_set, partition, model):
    
    #identifier = f"{endpoint}_{feature_set}_{model}_{partition}_mean" # for meaned preds
    identifier = f"{endpoint}_{feature_set}_{model}_{partition}"
    fp_in = f"{out_path}/{identifier}.feather"
    
    temp = read_data(fp_in)
    return temp

In [15]:
model_path = pathlib.Path(f"{experiment_path}/coxph/input")
models = [f.name for f in model_path.iterdir() if f.is_dir() and "ipynb_checkpoints" not in str(f)]
models

['ImageTraining_[]_ConvNeXt_MLPHead_predictions_cropratio0.3',
 'ImageTraining_[]_ConvNeXt_MLPHead_predictions_cropratio0.8',
 'ImageTraining_[]_ConvNeXt_MLPHead_predictions_cropratio0.5']

In [16]:
d = []

for endpoint in tqdm(endpoints):
    for feature_set in [
        "Age+Sex", "Retina", "Age+Sex+Retina",
        "SCORE2", "SCORE2+Retina",
        "ASCVD", "ASCVD+Retina",
        "QRISK3", "QRISK3+Retina"
                    ]:
        for partition in partitions:
            for model in models:
                try: 
                    temp = read_predictions(endpoint, feature_set, partition, model)
                    d.append({"endpoint": endpoint, "features":feature_set, "model":model, "partition":partition, "available": True})
                except:
                    d.append({"endpoint": endpoint, "features":feature_set, "partition":partition, "available": False})

  0%|          | 0/1171 [00:00<?, ?it/s]

In [19]:
pd.DataFrame.from_dict(d).groupby(["features"])["available"].sum().to_frame()

,available
features,
ASCVD,77286
ASCVD+Retina,77286
Age+Sex,77286
Age+Sex+Retina,77286
QRISK3,77286
QRISK3+Retina,77286
Retina,77286
SCORE2,77286
SCORE2+Retina,77286


In [20]:
pd.DataFrame.from_dict(d).groupby(["model", "features"])["available"].sum().to_frame()

available
model                                              features                 
ImageTraining_[]_ConvNeXt_MLPHead_predictions_c... ASCVD               25762
                                                   ASCVD+Retina        25762
                                                   Age+Sex             25762
                                                   Age+Sex+Retina      25762
                                                   QRISK3              25762
                                                   QRISK3+Retina       25762
                                                   Retina              25762
                                                   SCORE2              25762
                                                   SCORE2+Retina       25762
ImageTraining_[]_ConvNeXt_MLPHead_predictions_c... ASCVD               25762
                                                   ASCVD+Retina        25762
                                                   Age+Sex             25762
                                                   Age+Sex+Retina      25762
                                                   QRISK3              25762
                                                   QRISK3+Retina       25762
                                                   Retina              25762
                                                   SCORE2              25762
                                                   SCORE2+Retina       25762
ImageTraining_[]_ConvNeXt_MLPHead_predictions_c... ASCVD               25762
                                                   ASCVD+Retina        25762
                                                   Age+Sex             25762
                                                   Age+Sex+Retina      25762
                                                   QRISK3              25762
                                                   QRISK3+Retina       25762
                                                   Retina              25762
                                                   SCORE2              25762
                                                   SCORE2+Retina       25762

In [5]:
in_path = f"{experiment_path}/coxph/predictions"
prediction_paths = !ls $in_path
print(prediction_paths[0])
predictions = pd.Series(prediction_paths).str.split("_", expand=True)\
    .assign(path = prediction_paths)\
    .assign(endpoint = lambda x: x[0]+"_"+x[1])\
    .assign(score = lambda x: x[2])\
    .assign(model = lambda x: x[3]+"_"+x[4]+"_"+x[5]+"_"+x[6]+"_"+x[7]+"_"+x[8])\
    .assign(partition = lambda x: x[9].str.replace(".feather", "", regex=True).astype(int))\
    [["model", "endpoint", "score", "partition", "path"]].sort_values(["model", "endpoint", "score", "partition"]).reset_index(drop=True)
predictions

OMOP_4306655_Age+Sex_ImageTraining_[]_ConvNeXt_MLPHead_predictions_cropratio0.3_0.feather


,model,endpoint,score,partition,path
0,ImageTraining_[]_ConvNeXt_MLPHead_predictions_...,OMOP_4306655,ASCVD,0,OMOP_4306655_ASCVD_ImageTraining_[]_ConvNeXt_M...
1,ImageTraining_[]_ConvNeXt_MLPHead_predictions_...,OMOP_4306655,ASCVD,1,OMOP_4306655_ASCVD_ImageTraining_[]_ConvNeXt_M...
2,ImageTraining_[]_ConvNeXt_MLPHead_predictions_...,OMOP_4306655,ASCVD,2,OMOP_4306655_ASCVD_ImageTraining_[]_ConvNeXt_M...
3,ImageTraining_[]_ConvNeXt_MLPHead_predictions_...,OMOP_4306655,ASCVD,3,OMOP_4306655_ASCVD_ImageTraining_[]_ConvNeXt_M...
4,ImageTraining_[]_ConvNeXt_MLPHead_predictions_...,OMOP_4306655,ASCVD,4,OMOP_4306655_ASCVD_ImageTraining_[]_ConvNeXt_M...
...,...,...,...,...,...
695569,ImageTraining_[]_ConvNeXt_MLPHead_predictions_...,phecode_979,SCORE2+Retina,17,phecode_979_SCORE2+Retina_ImageTraining_[]_Con...
695570,ImageTraining_[]_ConvNeXt_MLPHead_predictions_...,phecode_979,SCORE2+Retina,18,phecode_979_SCORE2+Retina_ImageTraining_[]_Con...
695571,ImageTraining_[]_ConvNeXt_MLPHead_predictions_...,phecode_979,SCORE2+Retina,19,phecode_979_SCORE2+Retina_ImageTraining_[]_Con...
695572,ImageTraining_[]_ConvNeXt_MLPHead_predictions_...,phecode_979,SCORE2+Retina,20,phecode_979_SCORE2+Retina_ImageTraining_[]_Con...


In [6]:
predictions.to_feather(f"{experiment_path}/prediction_paths.feather")